# My Q\# notebook

> Superdense Coding and Deutsch-Josza were done by completing the Quanta Katas by [Microsoft](https://github.com/microsoft/QuantumKatas).

------

## Superdense Coding

We start with some superdense coding. Check 1.3.11 from my notes.

In [1]:
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Measurement;

newtype ProtocolMessage = (Bit1 : Bool, Bit2 : Bool);

operation CreateEntangledPair (q1 : Qubit, q2 : Qubit) : Unit is Adj {
    H(q1);
    CNOT(q1,q2);
}

operation EncodeMessageInQubit (qAlice : Qubit, message : ProtocolMessage) : Unit {
    if (message::Bit1) == false {
        if (message::Bit2) == true {
            X(qAlice);
        }
    } else {
        Z(qAlice);
        if (message::Bit2) == true {
            X(qAlice);
        }
    }
}

operation DecodeMessageFromQubits (qAlice : Qubit, qBob : Qubit) : ProtocolMessage {
    CNOT(qAlice,qBob);
    H(qAlice);
    mutable bit1 = false;
    mutable bit2 = false;
    if M(qAlice) == One {
        set bit1 = true;
    }
    if M(qBob) == One {
        set bit2 = true;
    }
    let mes = ProtocolMessage(bit1,bit2);
    ResetAll([qAlice,qBob]);
    return mes;
}

operation SuperdenseCodingProtocol (message : ProtocolMessage) : ProtocolMessage {
    use q = Qubit[2];
    CreateEntangledPair(q[0],q[1]);
    EncodeMessageInQubit(q[0],message);
    return DecodeMessageFromQubits(q[0],q[1]);
}

ProtocolMessage, CreateEntangledPair, EncodeMessageInQubit, DecodeMessageFromQubits, SuperdenseCodingProtocol

We now test our code by going through all permutations of $\{\text{true},\text{false}\}$

In [3]:
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Diagnostics;

operation TestSuperDenseCoding () : Bool {
    for n in 0 .. 3 {
        let data = ProtocolMessage(1 == n / 2, 1 == n % 2);
        
        for iter in 1 .. 100 {
            let result = SuperdenseCodingProtocol(data);
            
            Fact(result::Bit1 == data::Bit1 and result::Bit2 == data::Bit2, 
                    $"{data} was transfered incorrectly as {result}");
        }
    }
    return true;
}

TestSuperDenseCoding

In [4]:
%simulate TestSuperDenseCoding

True

---

## Deutsch-Josza

In [5]:
operation DeutschJozsaAlgorithm (N : Int, oracle : (Qubit[] => Unit)) : Bool {
    mutable isConstant = true;
    use x = Qubit[N];
    ApplyToEach(H,x);
    oracle(x);
    ApplyToEach(H,x);
    for q in 0 .. N-1 {
        if M(x[q]) != Zero {
            set isConstant = false; 
        }
    }
    return isConstant;
}

operation OracleZero (x : Qubit[]) : Unit {}

DeutschJozsaAlgorithm, OracleZero

Let us test our operation

In [6]:
operation TestDJ () : Bool {
    Fact(DeutschJozsaAlgorithm(4, OracleZero) == true, "f(x) = 0 not identified as constant");
    return true;
}

TestDJ

In [7]:
%simulate TestDJ

True

---

## Writing Circuit Models

The following is my implementation for Question 8, Chapter 2 from de Wolf's *Quantum Computing* lecture notes (or 1.4.7 from my notes).

In [8]:
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Diagnostics;
    
operation CheckIfAllZeros (q : Qubit[], b : Qubit) : Unit {
    // we start with using N auxiliary qubits -- where N is the number of how many qubits there are in q
    let N = Length(q);
    use d = Qubit[N];
    
    // we now apply the X gate to all qubits in q
    ApplyToEach(X,q);
    
    // if N = 1, then we apply a CNOT gate directly from q to d.
    // otherwise, we start with applying a CCNOT gate with the first two qubits in q 
    // being control and the first qubit in d being the target.
    if N > 1 {
        CCNOT(q[0],q[1],d[0]);
    } else {
        CNOT(q[0],d[0]);
    }
    
    // if N >= 3, then we apply a CCNOT looped over the range of N - 3, where we have
    // our control qubits being q[i+2] and d[i] and our target being d[i+1]
    for i in 0 .. N - 3 {
        CCNOT(q[i+2],d[i],d[i+1]);
    }
    
    // we now apply a CCNOT gate to our last qubit in d with the two before it being control qubits
    if N > 2 {
        CCNOT(d[N-3],d[N-2],d[N-1]);
    }
    
    // for the special case of N = 2, we apply a CNOT gate from the first qubit in d to the last qubit in d
    if N == 2 {
        CNOT(d[0],d[1]);
    }
    
    // this step is the step that either changes b to 1-b (if all q is zero) or keeps it as b (not all q is zero).
    // we do this by apply a CNOT gate from the last qubit in d to b
    CNOT(d[N-1],b);
    
    // we now return all our qubits to their original states (except for b)
    ApplyToEach(X,q);
    ResetAll(d);
}

CheckIfAllZeros

The following code is not necessary to run. This is just to have our output in bitstrings rather than the default.

In [9]:
%config dump.basisStateLabelingConvention = "Bitstring"

"Bitstring"

We now test our operation with the following code. Click run on %simulate to see the output. Feel free to edit the test operation.

> Remember to run CheckIfAllZeros operation and the TestCheckIfAllZeros operation before running the %simulation field.
> Uncomment the `DumpMachine();` lines to see the output of the qubits.

In [11]:
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Diagnostics;

operation TestCheckIfAllZeros () : Bool {
    use (qs, b) = (Qubit[2], Qubit());
    // X(qs[3]);
    // X(b);
    
    //DumpMachine();
    CheckIfAllZeros(qs,b);
    //DumpMachine();
    
    ResetAll(qs);
    return M(b) == One;
}

TestCheckIfAllZeros

In [12]:
%simulate TestCheckIfAllZeros

True

Run the following code to see the circuit diagram Q\# draws. The first $N$ qubits are $|q\rangle$, the $(N+1)^\text{th}$ qubit is $|b\rangle$, and everything that follows is an auxiliary qubit.

In [13]:
%trace TestCheckIfAllZeros

Microsoft.Quantum.IQSharp.Jupyter.DisplayableHtmlElement

---